In [1]:
import sys
sys.path.append("/home/tbaweja/wproj/")

In [2]:
from transformers import AutoTokenizer, CLIPTextModel
from tqdm import tqdm
import torch

In [3]:
device = "cuda:3" if torch.cuda.is_available() else "cpu"

In [4]:
from dataloader.coco_dataloader import COCODataset
dataset = COCODataset()

In [5]:
captions = [dataset[i]["caption"] for i in range(len(dataset))]

In [12]:
train_captions = captions[:58000]
val_captions = captions[58000:]

Loading pretrained encoder model of CLIP to produce textual embeddings

In [8]:
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
import os
if not(os.path.exists("./text_embeddings_train")):
    os.makedirs("./text_embeddings_train")

if not(os.path.exists("./text_embeddings_val")):
    os.makedirs("./text_embeddings_val")
    

Saving textual embeddings in torch format

In [21]:
for i, caption in enumerate(tqdm(train_captions)):
    inputs = tokenizer(caption, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    pooled_output = outputs.pooler_output 

    torch.save(pooled_output, f'./text_embeddings_train/{str(i+1)}.pt')

100%|██████████| 58000/58000 [33:34<00:00, 28.79it/s]


In [22]:
for i, caption in enumerate(tqdm(val_captions)):
    inputs = tokenizer(caption, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    pooled_output = outputs.pooler_output 

    torch.save(pooled_output, f'./text_embeddings_val/{str(i+1)}.pt')

100%|██████████| 6783/6783 [03:48<00:00, 29.63it/s]
